In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import StratifiedKFold,KFold
from datasets import Dataset 
from datasets import Features, ClassLabel, Sequence, Value
from datasets import load_from_disk, load_dataset, DatasetDict
from pandasql import sqldf
from transformers import AutoTokenizer, DataCollatorWithPadding

from utils import find_model_with_pos, find_error_with_reason_with_pos, find_error_with_reason, find_model

In [2]:
import numpy as np 
import pandas as pd
from collections import Counter
from pandasql import sqldf
import re
import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'
import joblib
import json 
import jieba 
import copy

from openai import AzureOpenAI
import matplotlib.pyplot as plt 
from sentence_transformers import SentenceTransformer
from FlagEmbedding import FlagReranker
from transformers import AutoTokenizer
from sklearn.metrics import average_precision_score

In [3]:
# pandasql查询函数需要的环境
pysqldf = lambda q: sqldf(q, globals())

In [4]:
# 原始数据处理
def format_model2(x):
    model_list = x.split(',')
    model_list = [i.strip().lower() for i in model_list]
    new_list = [model_list[0]]
    i = 1
    while i < len(model_list):
        if (i != len(model_list) - 1) and (model_list[i-1] == model_list[i]):
            new_list.append(model_list[i]+model_list[i+1])
            if i < len(model_list) - 1:
                i += 2
            else:
                break
        elif (model_list[i][:3] == "上下水") or (model_list[i][:3] == "air") or (model_list[i][:3] == "pro") or (model_list[i][:4] == "pure"):
            for j in range(len(new_list)-1, -1, -1):
                if model_list[i-1] == new_list[j]:
                    new_list.pop(j)
                    break
            new_list.append(model_list[i-1]+model_list[i])
            i += 1
        elif (i != len(model_list) - 1) and (model_list[i-1] != model_list[i]):
            new_list.append(model_list[i])
            i += 1
        else:
            new_list.append(model_list[i])
            break
    return new_list

def format_all_models(x, dim_df):
    new_list = []
    for i in x:
        if i.find("全型号") >= 0:
            end_idx = i.find("全型号")
            name = i[:end_idx]
            new_list += [j for j in dim_df[dim_df['cat_name'] == name].model.tolist() if j not in x]
        else:
            new_list.append(i)
    return new_list

def format_series(x, dim_df):
    def contains_chinese(s):
        return re.search('[\u4e00-\u9fff]', s) is not None
    new_list = []
    for i in x:
        if i.find("系列") >= 0:
            end_idx = i.find("系列")
            name = i[:end_idx]
            new_list += [j for j in dim_df[(dim_df.model.str.find(name)>=0) & (
                dim_df.model.apply(lambda x: not contains_chinese(x)))].model.tolist() if j not in x]
            new_list += [i]
        else:
            new_list.append(i)
    return new_list

In [5]:
oot = pd.read_excel("/data/dataset/kefu/Rafe Copy 国内客服助手（生产环境）_测试用数据表_表格.xlsx")
oot = oot.drop_duplicates("问题")
oot = oot[["编号", "问题", "匹配的标准问题编号"]].rename(columns={"编号": "qa_id",
                                                      "问题": "question", 
                                                      "匹配的标准问题编号": "gt_qa_id"})

In [454]:
oot.gt_qa_id.isnull().sum() / oot.shape[0]

0.3018867924528302

In [455]:
oot = oot[oot["gt_qa_id"].notnull()]

In [456]:
oot["gt_qa_id"] = oot["gt_qa_id"].apply(lambda x: x.replace("，", ",").replace("\n", ""))

In [457]:
dim_df = pd.read_csv("/data/dataset/kefu/dim_df20240620.csv")
all_model_list = dim_df["model"].tolist()

In [458]:
dim_df_old = pd.read_csv("/data/dataset/kefu/dim_df20240315.csv")
all_model_list_old = dim_df_old["model"].tolist()

In [459]:
oot["model_list_old"] = oot["question"].apply(lambda x: find_model(x, all_model_list_old))
oot["model_list"] = oot["question"].apply(lambda x: find_model(x, all_model_list))

In [460]:
oot["new_model"] = oot[["model_list_old", "model_list"]].apply(lambda x: set(x.model_list_old) != set(x.model_list), axis=1)

In [461]:
oot["new_model"].value_counts()

new_model
False    591
True       1
Name: count, dtype: int64

In [6]:
df1 = pd.read_csv("/data/dataset/kefu/database20240620.csv")
df2 = pd.read_csv("/data/dataset/kefu/database20240506.csv")

In [7]:
df2.qa_id.max()

'ICWIKI202403081800'

In [463]:
problem_all = []
for i in range(oot.shape[0]):
    problem = []
    gt_qa_id = oot.iloc[i].gt_qa_id
    for id in gt_qa_id.split(","):
        if id not in df1.qa_id.tolist():
            problem.append(id)
    problem_all.append(",".join(problem))

In [464]:
oot["problem"] = problem_all

In [466]:
oot[oot.problem==""].shape[0] / oot.shape[0]

0.7466216216216216

In [483]:
def find_real(x):
    if x.problem == "":
        return False
    else:
        return len(set(x.gt_qa_id.split(","))) == len(set(x.problem.split(",")))

oot["real_problem"] = oot[["gt_qa_id", "problem"]].apply(
    lambda x: find_real(x), axis=1)

In [485]:
oot[oot.real_problem==True].shape[0] / oot.shape[0]

0.10641891891891891

In [486]:
oot.to_csv("/root/PycharmProjects/pya/data/oot20240620.csv", index=None)

In [378]:
dim_df = pd.read_excel("/data/dataset/kefu/国内客服助手（生产环境） 副本.xlsx", sheet_name="品类与型号关系表")

In [404]:
dim_df = dim_df.rename(columns={"编号": "model_id", 
                                "品类": "cat_name", 
                                "型号": "model", 
                                "是否已生效": "effective"})
dim_df["model"] = dim_df["model"].str.lower()
dim_df['model'] = dim_df['model'].apply(lambda x: ''.join(x.split(' ')))
all_model_list = dim_df["model"].tolist()

In [541]:
df1_old = pd.read_csv("/root/PycharmProjects/pya/data/database20240506.csv")

In [542]:
df1_old.shape

(3016, 13)

In [543]:
new = pd.read_excel("/data/dataset/kefu/国内客服助手（生产环境） 副本.xlsx", sheet_name="知识库数据")
new = new[["编号", "语音回复内容"]].rename(columns={"编号": "qa_id", "语音回复内容": "answer_voice"})

In [544]:
df1_old = pd.merge(left=df1_old, right=new, left_on="qa_id", right_on="qa_id", how="left")

In [545]:
df1_old.shape

(3016, 14)

In [548]:
df1_old.answer_voice.isnull().sum()

406

In [547]:
df1_old.loc[df1_old.answer_voice.notnull(), "answer"] = df1_old.loc[df1_old.answer_voice.notnull(), "answer_voice"]

In [549]:
df1_old.answer.isnull().sum()

0

In [550]:
df1_old = df1_old.drop("answer_voice", axis=1)

In [552]:
df1_old.to_csv("/root/PycharmProjects/pya/data/database20240506_answervoice.csv", index=None)

In [416]:
df1 = pd.read_excel("/data/dataset/kefu/国内客服助手（生产环境） 副本.xlsx", sheet_name="知识库数据")

In [417]:
df1.shape

(3141, 16)

In [418]:
df1 = df1[df1["问题"].notnull()]

In [419]:
df1.shape

(3140, 16)

In [420]:
query = f"""
select 
    *
    ,row_number() over (partition by 适配机型, 问题 order by 变更日期) as count_no
    ,count(*) over (partition by 适配机型, 问题) as count_unique
from 
    df1
"""

# 使用pysqldf执行SQL查询
df1 = pysqldf(query)

In [421]:
df1 = df1[(df1['count_unique']==df1['count_no'])]

In [422]:
df1.shape

(3102, 18)

In [423]:
df1 = df1.drop(["count_no", "count_unique"], axis=1)

In [424]:
rename = {'编号': "qa_id", 
          '类型': "qa_type", 
          '问题': "question", 
          '语音回复内容': "answer", 
          '适配机型': "model", 
          '是否已生效': "effective", 
          '更新人': "update_by", 
          '变更日期': "update_time"}
df1 = df1.rename(columns=rename)
df1['question'] = df1['question'].apply(lambda x: x.strip('\r'))

In [425]:
df1["model_list"] = df1["model"].apply(lambda x: format_model2(x))
df1["model_list"] = df1['model_list'].apply(lambda x: format_all_models(x, dim_df))
df1["model_list"] = df1['model_list'].apply(lambda x: format_series(x, dim_df))
df1["model_list"] = df1["model_list"].apply(lambda x: ','.join(x))

In [426]:
df1 = df1[["qa_id", "qa_type", "question", "answer", "model", "model_list", "update_time"]]

In [427]:
df1.shape

(3102, 7)

In [428]:
temp = df1.copy()
temp["model_list"] = temp["model_list"].apply(lambda x: x.split(','))
temp_exploded = temp.explode("model_list")

query = f"""
select 
    a.*
    ,b.model_id
    ,b.cat_name
from 
    temp_exploded a 
left join 
    dim_df b
on 
    a.model_list = b.model
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

query = f"""
select 
    qa_id
    ,group_concat(model_id) as model_id
    ,group_concat(cat_name) as cat_name
from 
    temp_exploded
group by 
    qa_id
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

query = f"""
select 
    a.*
    ,b.model_id
    ,b.cat_name
from 
    df1 a 
left join 
    temp_exploded b
on
    a.qa_id = b.qa_id
"""

# 使用pysqldf执行SQL查询
df1 = pysqldf(query)

In [429]:
model_list = dim_df.model.unique().tolist()
for i in range(df1.shape[0]):
    for model in df1.model_list.iloc[i].split(","):
        if model not in model_list:
            print(df1.qa_id.iloc[i], model)

In [430]:
df1["error_list"] = df1["question"].apply(lambda x: ",".join(find_error_with_reason(x)))
df1["answer"] = df1["answer"].apply(lambda x: x.replace("\r", ""))

In [431]:
df1.shape

(3102, 10)

In [396]:
df1.to_csv("/data/dataset/kefu/database20240620.csv", index=None)

In [397]:
dim_df.to_csv("/data/dataset/kefu/dim_df20240620.csv", index=None)

In [240]:
temp = df1.copy()

In [241]:
temp["model_list"] = temp["model_list"].apply(lambda x: x.split(','))
temp_exploded = temp.explode("model_list")

In [242]:
temp_exploded.shape

(20029, 7)

In [243]:
query = f"""
select 
    a.*
    ,b.model_id
    ,b.cat_name
from 
    temp_exploded a 
left join 
    dim_df b
on 
    a.model_list = b.model
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

In [244]:
temp_exploded.shape

(20029, 9)

In [245]:
query = f"""
select 
    *
    ,row_number() over (partition by question, model_list order by update_time) as count_no
    ,count(*) over (partition by question, model_list) as count_unique
from 
    temp_exploded
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

In [246]:
temp_exploded = temp_exploded[(temp_exploded['count_unique']==temp_exploded['count_no'])]
temp_exploded = temp_exploded.drop(["count_no", "count_unique"], axis=1)

In [247]:
temp_exploded.shape

(19920, 9)

In [248]:
def group_table(x):
    qa_id = x["qa_id"].tolist()
    qa_type = x["qa_type"].tolist()
    answer = x["answer"].tolist()
    model_list = x["model_list"].tolist()
    model = x["model"].tolist()
    model_id = x["model_id"].tolist()
    cat_name = x["cat_name"].tolist()
    return pd.Series([qa_id, qa_type, answer, model_list, model, model_id, cat_name],
             index=["qa_id", "qa_type", "answer", "model_list", "model", "model_id", "cat_name"])

In [249]:
temp_exploded2 = temp_exploded.groupby("question").apply(group_table)

/tmp/ipykernel_1651295/2099210936.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_exploded2 = temp_exploded.groupby("question").apply(group_table)


In [292]:
temp_exploded2["index"] = range(temp_exploded2.shape[0]

In [293]:
for col in ["qa_id", "qa_type", "answer", "model_list", "model", "model_id", "cat_name"]:
    temp_exploded2[col] = temp_exploded2[col].apply(lambda x: json.dumps(x, ensure_ascii=False))

In [296]:
temp_exploded2.shape

(2476, 8)

In [300]:
model_list = dim_df.model.unique().tolist()
for i in range(temp_exploded2.shape[0]):
    for model in json.loads(temp_exploded2.model_list.iloc[i]):
        if model not in model_list:
            print(temp_exploded2.qa_id.iloc[i], model)

In [307]:
temp_exploded2 = temp_exploded2.reset_index(drop=False)

In [309]:
temp_exploded2["error_list"] = temp_exploded2["question"].apply(
    lambda x: json.dumps(find_error_with_reason(x), ensure_ascii=False))

In [310]:
temp_exploded2.shape

(2476, 10)

In [311]:
temp_exploded2.to_csv("/data/dataset/kefu/database20240620_grouped.csv", index=None)